## 3. Gated Convolutional Network for emotion recognition

Import the required modules.

In [ ]:
from keras.models import Sequential
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing import sequence
from keras.layers import Conv1D, Input, MaxPooling1D
from keras.layers import add, multiply
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Model, Sequential, load_model
from keras.optimizers import Adam, SGD
from keras.utils import to_categorical
import os
import numpy as np
import csv
import random
from keras.layers import LeakyReLU

In [ ]:
data_path = "/work/peiyun/data"

### 3.1. Set parameters for GCNN

In [ ]:
avg_len = 0
max_len = 0
count = 0
for data_type in ["train", "dev", "test"]:
    for config in ["IS09_emotion", "IS10_paraling"]:
        d_path = os.path.join(data_path, "output", data_type, config)
        for filename in os.listdir(d_path):
            if filename.endswith(".csv"):
                with open(os.path.join(d_path, filename) , mode='r') as f:
                    row_num = sum(1 for i in f)
                    avg_len += row_num
                    if row_num > max_len:
                        max_len = row_num
                    f.close()
                count += 1
avg_len /= count
print avg_len
print max_len

In [ ]:
# Parameters
MAX_LEN = 3450 # 3449 (for each utterance)
BATCH_SIZE = 4
NUM_EPOCH = 32
KERNEL_SIZE = 2

IS_DILATED = False
DILATION_RATE = 1
N_STACK = 3

### 3.2. Extracting the matrices and labels for all data types and store as numpy files

Function for obtaining label according to the label.csv file.

In [ ]:
# Get the ground_truth label number of the file
def get_label(filename):
    label_file = os.path.join(data_path, "label.csv")
    
    with open(label_file, mode = "r") as f:
        reader = csv.reader(f)
        for row in reader:
            name, session, label, dims, data_type, transcript = row
            # print filename
            if name == filename:
                return label

Function for extracting the corresponding normalised input and the ground truth output of a file.

In [ ]:
def extract_Xy(file_path):
    
    # Obtain dataframe for the csv file (one utterance/instance)
    df = pd.read_csv(file_path, sep = ";")
    
    # Get filename
    filename = df["name"][0][1:-1]  # [1:-1] for removing single quotation marks
    
    # Clean unnecessary columns
    df = df.drop(columns = ["name", "frameTime"])
    
    # Normalise data
    x = df.values                # dataframe to a numpy array
    min_max_scalar = MinMaxScaler()  
    x_scaled = min_max_scalar.fit_transform(x)     # scaling each feature (each column) to range: (0,1)
    df = pd.DataFrame(x_scaled)  # numpy array back to dataframe
    
    
    # Obtain matrix X and label y, X_dim: timesteps x features
    x_data = sequence.pad_sequences(df.values.T, padding = "post", dtype = "float64", maxlen = MAX_LEN).T
    
    y_data = get_label(filename)
    
    return x_data, y_data

In [ ]:
extract_Xy("/work/peiyun/data/output/train/IS09_emotion/Ses05M_script03_2_M040_IS09_emotion.csv")

In [ ]:
# IS09 has 32 LLDs and IS10 has 76 LLDs
def get_feature_num(config):
    
    # enter the folder for the config under train (can also use test/dev, same result)
    path = os.path.join(data_path, "output", "train", config)
    
    # enter any config folder (nums of files are the same for all configs)
    for filename in os.listdir(path):
        if filename.endswith(".csv"):
            path = os.path.join(path, filename)
            break
    
    # Obtain dataframe for the csv file (one utterance/instance)
    df = pd.read_csv(path, sep = ";")
    
    # Clean unnecessary columns
    df = df.drop(columns = ["name", "frameTime"])

    return len(df.columns)

Extract the required matrix and label pairs for each data type and store as numpy files. 
<br>
NOTE: np.load(filename.npy) to load data

In [ ]:
def get_file_num(data_type):
    
    # enter the folder for the data_type
    path = os.path.join(data_path, "output", data_type)
    
    # enter any config folder (nums of files are the same for all configs)
    for config in os.listdir(path):
        path = os.path.join(path, config)
        break
        
    return len(os.listdir(path))/2   # only half are csv files

In [ ]:
temp_dir = os.path.join(data_path, "temp")
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)

In [ ]:
label_dict = {"ang": 0,
              "exc": 1,
              "fru": 2,
              "hap": 3,
              "neu": 4,
              "sad": 5,
              "sur": 6}

In [ ]:
# save as numpy file
np.save(os.path.join(data_path, "label_dict.npy"), label_dict)

# # loading numpy file
# label_dict = np.load(os.path.join(data_path, "label_dict.npy")).item()

In [ ]:
numpy_dir = "numpy_var"

In [ ]:
if not os.path.exists(os.path.join(data_path, numpy_dir)):
    os.makedirs(os.path.join(data_path, numpy_dir))

In [ ]:
for config in ["IS09_emotion", "IS10_paraling"]:
    
    for data_type in ["train", "test", "dev"]:

        # temp dat file for storing X
        X_data_filename = os.path.join(temp_dir, data_type + ".dat")
        
        # Obtain the number of features and number of files
        feature_num = get_feature_num(config)
        file_num = get_file_num(data_type)
        
 
        # initialise lists (using memory-map for accessing small segments of large files on disk)
        X = np.memmap(X_data_filename, dtype='float64', mode='w+', shape=((file_num, MAX_LEN + KERNEL_SIZE - 1, feature_num)))
        Y = []

        # path for the directory of each data_type
        path = os.path.join(data_path, "output", data_type, config)
        
        i = 0

        # iterate through all files
        for filename in os.listdir(path):

            # only interested in csv files
            if not filename.endswith(".csv"):
                continue

            # extract data
            x_data, y_data = extract_Xy(os.path.join(path, filename))
            Y.append(label_dict[y_data])
            
            # Padding zeros to the beginning of the sequences 
            # with kernel_size - 1 elements to prevent kernels from seeing the future context
            zeros = np.zeros((KERNEL_SIZE - 1, feature_num))
            x_data = np.concatenate((zeros, x_data), axis = 0)
            X[i] = x_data
            
            i += 1
            
            if i%100 == 0:
                print "num of file processed: " + str(i)

        # save as numpy file
        np.save(os.path.join(data_path, numpy_dir, data_type + "_" + config + "_X.npy"), X)
        np.save(os.path.join(data_path, numpy_dir, data_type + "_" + config + "_Y.npy"), Y)
        
        # delete temp data
        del X
        
# delete temp dir
shutil.rmtree(temp_dir)

### 3.3. Read all the required data from the numpy files.

Function for obtaining the required data for the configuration. 

In [ ]:
def get_variables(config):
    
    # variables
    X_train = np.load(os.path.join(data_path, numpy_dir, "train_" + config + "_X.npy"), mmap_mode = "r")
    y_train = np.load(os.path.join(data_path, numpy_dir, "train_" + config + "_Y.npy"))
    
    X_test = np.load(os.path.join(data_path, numpy_dir, "test_" + config + "_X.npy"), mmap_mode = "r")
    y_test = np.load(os.path.join(data_path, numpy_dir, "test_" + config + "_Y.npy"))
    
    X_dev = np.load(os.path.join(data_path, numpy_dir, "dev_" + config + "_X.npy"), mmap_mode = "r")
    y_dev = np.load(os.path.join(data_path, numpy_dir, "dev_" + config + "_Y.npy"))
    
    return X_train, y_train, X_test, y_test, X_dev, y_dev

Try reading all data for config: IS09_emotion.

In [ ]:
X_train, y_train, X_test, y_test, X_dev, y_dev = get_variables("IS09_emotion")

In [ ]:
X_train.shape

### 3.4. Define pre-activation residual block

In [ ]:
def residual_block(input_layer):   # containing: convolution + gated linear unit
    
    # Obtain convolution layer (1D, temporal convolution) by creating a convolution kernel
    tanh_out = Conv1D(NUM_FILTER, 
                      kernel_size = KERNEL_SIZE,  # length of 1D convolution window = 2
                      kernel_initializer= "random_uniform",  # initialization of filters
                      dilation_rate = DILATION_RATE,
                      padding = "same")(input_layer)   # Conv1D “depending” on the input layer

    # Normalise the result after Conv1D
    tanh_out = BatchNormalization()(tanh_out)

    # Obtain the convolutional layer with sigmoid transformation (activation)
    sigmoid_out = Conv1D(NUM_FILTER, 
                         kernel_size = KERNEL_SIZE,
                         kernel_initializer='random_uniform',
                         dilation_rate = DILATION_RATE,
                         padding = "same")(input_layer)
    
    # Normalise the layer
    sigmoid_out = BatchNormalization()(sigmoid_out)
    
    # Activation function for the layer
    sigmoid_out = Activation("sigmoid")(sigmoid_out)

    # Element-wise multiplication
    merged = multiply([tanh_out, sigmoid_out])
    
#     # Max Pooling for the merged result
#     merged = MaxPooling1D(pool_size = 2)(merged)
    
    # Adding input to output and apply relu
    merged = add([input_layer, merged])  # addition
    gated_cnn = Activation("relu")(merged)
    
    return merged

### 3.5. Define a function for generating the whole model

In [ ]:
def generate_model(input_shape):  # input shape: num_files x time x LLD *****
    
    input_layer = Input(shape = input_shape)  # time x LLD
    gated_cnn = residual_block(input_layer)  # first block
    
    for i in range(0, N_STACK - 1):  # the rest blocks
        gated_cnn = residual_block(gated_cnn)
    
    # Flattening
    gated_cnn = Flatten()(gated_cnn)
    
    # Fully Connected Layer
    gated_cnn = Dense(256, kernel_initializer = "random_uniform")(gated_cnn) 
    gated_cnn = BatchNormalization()(gated_cnn)
    gated_cnn = Activation("relu")(gated_cnn)
    gated_cnn = Dropout(0.5)(gated_cnn)
    
#     # Fully Connected Layer then sigmoid
#     gated_cnn = Dense(11, activation = "softmax")(gated_cnn)

    # Fully Connected Layer then sigmoid
    gated_cnn = Dense(7, activation = "softmax")(gated_cnn)


    all_model = Model(inputs = input_layer, outputs = gated_cnn) # This model will include all layers required in the 
                                                                 # computation of "outputs" given "inputs"

    all_model.compile(loss = "categorical_crossentropy",
                      optimizer = "adam",
                      metrics=["accuracy"])

#     all_model.summary()
    
    return all_model

### 3.6. Training a GCNN model with the feature configuration

Function for training a GCN model with the config.

In [ ]:
def training(model_name):
    
    X_train, y_train, X_test, y_test, X_dev, y_dev = get_variables(model_name)
    
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    y_dev = to_categorical(y_dev)
    
    model = generate_model(X_train.shape[1:])  # model after compilationhttps://www.youtube.com/watch?v=3jWRrafhO7M&start_radio=1&list=RD3jWRrafhO7M
    
    # fitting training data
    model.fit(X_train, y_train,
              batch_size = BATCH_SIZE,
              epochs = NUM_EPOCH,
              validation_data = (X_dev, y_dev))
    
    # saving model
    model.save(model_name + "3.model")  # save model
    
#     # load model
#     new_model = load_model("epic_num_reader.model")

Begin training model by calling the training function.

In [ ]:
NUM_FILTER = 32

In [ ]:
# load model
new_model = load_model("IS09_emotion3.model")

In [ ]:
X_test.shape

In [ ]:
loss, accuracy = new_model.evaluate(X_test, to_categorical(y_test))
print "loss: " + str(loss)
print "accuracy: " + str(accuracy)

In [ ]:
training("IS09_emotion")

In [ ]:
NUM_FILTER = 76

In [ ]:
training("IS10_paraling")

Try to use 4 categories: anger, excitement(happiness), neutral, and sadness

In [ ]:
# loading numpy file
label_dict = np.load(os.path.join(data_path, "label_dict.npy")).item()

In [ ]:
label_dict

In [ ]:
def training2(X_train, y_train, X_test, y_test, X_dev, y_dev):
    
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    y_dev = to_categorical(y_dev)
    
    model = generate_model(X_train.shape[1:])  # model after compilationhttps://www.youtube.com/watch?v=3jWRrafhO7M&start_radio=1&list=RD3jWRrafhO7M
    
    # fitting training data
    model.fit(X_train, y_train,
              batch_size = BATCH_SIZE,
              epochs = NUM_EPOCH,
              validation_data = (X_dev, y_dev))
    
    # saving model
    model.save(model_name + "_4_category.model")  # save model
    
#     # load model
#     new_model = load_model("epic_num_reader.model")

In [ ]:
print X_train.shape
print y_train.shape

In [ ]:
y_train

In [ ]:
selected_emo = [0,1,3,4,5]  # combine 1 and 3 latter, converting all 3's to 1's

In [ ]:
new_X_train = []
new_y_train = []
for i in range(len(X_train)):
    if y_train[i] in selected_emo:
        new_X_train.append(X_train[i])
        if y_train[i] == 3:
            new_y_train.append(1)
        else:
            new_y_train.append(y_train[i])

In [ ]:
len(new_X_train)